In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.metrics import mean_squared_error


Bad key text.latex.unicode in file /Users/caochenjie/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /Users/caochenjie/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key nbagg.transparent in file /Users/caochenjie/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 426 ('nbagg.transparent: True')
You probably need to get an updated matplotlibr

In [2]:
from torch import nn
import torch
from torch.nn  import functional as F
import torch.optim as  optim 
if torch.cuda.is_available():  
  dev = "cuda:0" 
  print("gpu up")
else:  
  dev = "cpu"  
device = torch.device(dev)

In [3]:
ratings = pd.read_csv("./archive/rating.csv")
anime = pd.read_csv("./archive/anime.csv")

In [4]:
"""
we will replace null values from categorical columns with "NULL"(like it's a new category) 
and for the rating column we will use the mean
"""
anime.genre.fillna("NULL",inplace=True)
anime.type.fillna("NULL",inplace=True)
anime.rating.fillna(anime.rating.mean(),inplace=True)

In [5]:
"""
these are the animes that we don't have information about, they are just 3 and
they have only 10 combinations(which means they contribute with only 10 samples) with the users but they will produce null values  for other features  
we will drop them  
"""
badAnimes = [x  for x in ratings.anime_id.unique() if x not in anime.anime_id.unique() ] 
badIndecies = ratings[ratings.anime_id.isin(badAnimes) ].index
ratings.drop(badIndecies,inplace=True)

In [6]:
"""
on how weird this processing seems, but it had to be like that for memory efficiency

"""

ratingsMean = anime.rating.mean()
membersMean = anime.members.mean()
base = anime.copy() 

base.drop(["episodes","name"],axis=1,inplace=True) #dropped episodes  many null values 
# and will cause a false  relationship as there are many movies (1 episode)
base.rename(columns={"rating":"avgRating"},inplace=True)

base.genre = base.genre.apply(lambda x: " ".join(x.split(" ")).split(", ")) #to remove unnecessary spaces
genres = [] # generating the genres data
for i in range(len(base)):
    for g in base.genre[i]:
        if g in genres:
            base["genre_"+g][i] = 1
        else:
            base["genre_"+g] = 0
            base["genre_"+g][i] = 1
            genres.append(g)



base = pd.concat([base,pd.get_dummies(base['type'], prefix='type',dummy_na=True)],axis=1).drop(['type'],axis=1)


ratings =ratings[["rating","user_id","anime_id"]] # rearranging columns
base= pd.merge(ratings,base,how="left",on="anime_id")
ratings = None


base.avgRating.fillna(ratingsMean,inplace=True)
base.members.fillna(membersMean,inplace=True)
base.drop(["genre"],axis=1,inplace=True)


base.fillna(0,inplace=True) # for any 

/Users/caochenjie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/caochenjie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
import random
random.seed(42)

"""
we will set the productionData to the data where the users didn't rate the anime
and we will use 500000 rows for validation

"""


productionData= base[base.rating == -1]
base.drop(base[base.rating == -1].index,inplace=True)
base = base.sample(frac=1).reset_index(drop=True)
validation = base.iloc[:500000]
train = base.iloc[500000:]
base = None # for reducing memory usage (we won't need this df again)

In [8]:
train.shape

(5837239, 57)

In [9]:
membersEnc = MinMaxScaler()
avgRatingEnc = MinMaxScaler()

animeEncoder = LabelEncoder()

"""
since the anime ids are not consecutive numbers, we will have to label encode them for the embedding layer 
"""

anime["anime_id"] = animeEncoder.fit_transform(anime["anime_id"])



train["anime_id"] = animeEncoder.transform(train["anime_id"])
train["avgRating"] = avgRatingEnc.fit_transform(train["avgRating"].to_numpy().reshape(-1,1))[:,0]
train["members"] = membersEnc.fit_transform(train["members"].to_numpy().reshape(-1,1))[:,0]

validation["anime_id"] = animeEncoder.transform(validation["anime_id"])
validation["avgRating"] = avgRatingEnc.transform(validation["avgRating"].to_numpy().reshape(-1,1))[:,0]
validation["members"] = membersEnc.transform(validation["members"].to_numpy().reshape(-1,1))[:,0]


productionData["anime_id"] = animeEncoder.transform(productionData["anime_id"])
productionData["avgRating"] = avgRatingEnc.transform(productionData["avgRating"].to_numpy().reshape(-1,1))[:,0]
productionData["members"] = membersEnc.transform(productionData["members"].to_numpy().reshape(-1,1))[:,0]

/Users/caochenjie/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [12]:
train.head()

,rating,user_id,anime_id,avgRating,members,genre_Drama,genre_Romance,genre_School,genre_Supernatural,genre_Action,...,genre_Yaoi,genre_Yuri,type_Movie,type_Music,type_NULL,type_ONA,type_OVA,type_Special,type_TV,type_nan
500000,10,69497,11485,0.776119,0.028436,0,0,1,1,1,...,0,0,0,0,0,1,0,0,0,0
500001,9,20129,1718,0.796472,0.301616,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
500002,8,44349,7157,0.643148,0.054841,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
500003,9,51219,3614,0.716418,0.240082,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
500004,10,10348,4695,0.742198,0.043094,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
